In [ ]:
import pandas as pd

In [ ]:
pip install google-play-scraper

In [ ]:
# Escolhemos a biblioteca google_play_scrapper para fazer o scrapping do google play

from google_play_scraper import app 

In [ ]:
#A função reviews_all retorna todas as avaliações do app.

from google_play_scraper import Sort, reviews_all 

In [ ]:
# Com o ID do AG0 na Google Play Store podemos utilizar a função reviews_all

avaliacoes = reviews_all( 'com.safra.safrawallet', lang = 'pt', country = 'br', sort = Sort.MOST_RELEVANT, sleep_milliseconds = 0)

In [ ]:
df = pd.DataFrame(avaliacoes)

In [ ]:
import nltk #NLTK é uma biblioteca usada para trabalhar com dados de linguagem humana.

nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
# A função word_tokenize é utilizada para dividir uma determinada frase em palavras

df['conteudo'] = df.apply(lambda row: nltk.word_tokenize(row['conteudo']), axis=1) 

In [ ]:
# Remover as "Stopwords", palavras comumente usadas, como "o", "um", "uma)

import re
from nltk.corpus import stopwords
nltk.download('stopwords') 
language = 'portuguese'

stopwords = stopwords.words(language)
stopwords = list(set(stopwords))

In [ ]:
# Função para remover as Stopwords das palavras tokenizadas

def remover_stopwords(palavras):
    lista_palavras = []
    for palavra in palavras:
        if palavra not in stopwords:
            lista_palavras.append(palavra)
    return lista_palavras

In [ ]:
# Função para mudar palavras para lowercase

def para_minuscula(palavras):
    """converter todos os caracteres para lowercase"""
    lista_palavras = []
    for palavra in palavras:
        nova_palavra = palavra.lower()
        lista_palavras.append(nova_palavra)
    return lista_palavras

In [ ]:
# Função para remover pontuação

def remover_pontuacao(words):
    """remover pontuação"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [ ]:
# Função para aplicar todas as funções acima.

def normalize(words):
    words = para_minuscula(words)
    words = remover_pontuacao(words)
    words = remover_stopwords(words)
    
    return ' '.join(words)

In [ ]:
# Aplicar nossa limpeza de dados em todas as linhas da coluna conteudo

df['conteudo'] = df.apply(lambda row: normalize(row['conteudo']), axis=1)

In [ ]:
# Vamos utilizar o documento 'sentimentos.txt' que é um um dicionário de sentimos open source obtida no github.

sentimentos = open('sentimentos.txt')

In [ ]:
# Obter a contagem de palavras das reviews do AG0 que estejam contidadas no documento sentimentos.txt

d = dict() 
for linha in df['conteudo']: 
    linha = linha.strip() 
    linha = linha.lower()
    palavras = linha.split(" ") 
    
    for palavra in palavras: 
      if palavra in d: 
              d[palavra] = d[palavra] + 1
      else: 
              d[palavra] = 1
d = sorted(d.items(), key=lambda x: x[1], reverse = True)
for i in d:
  if i[0] in sentimentos:
    print(i)

In [ ]:
dic_palavra_polaridade = {}
for i in sentimentos.readlines():
  pos_ponto = i.find('.')
  palavra = (i[:pos_ponto])
  pol_pos = i.find('POL')
  polaridade = (i[pol_pos+7:pol_pos+9]).replace(';', '')
  dic_palavra_polaridade[palavra] = polaridade

In [ ]:
def Score_sentimento(frase):
    frase = frase.lower()
    l_sentimento = []
    for p in frase.split():
        l_sentimento.append(int(dic_palavra_polaridade.get(p, 0)))
    score = sum(l_sentimento)
    if score > 0:
        return 'Pos {} '.format(score)
    elif score == 0:
        return 'Neu {} '.format(score)
    else:
        return 'Neg {}'.format(score)

In [ ]:
df['sentimento'] = df.apply(lambda row: Score_sentimento(row['content']), axis=1)
df['Score_Sentimento'] = df['sentimento'].str.slice(-2)
df['Score_Sentimento'] =df['Score_Sentimento'].astype(int)
df['Sent'] = df['sentimento'].str.slice(0,-3)

In [ ]:
df.groupby('Score_Sentimento').count()

In [ ]:
content = df['content']
df['content']

In [ ]:
all_content = " ".join(c for c in content)
all_content

In [ ]:
print(palavra)

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
stopwords.update(["pra", "app", "aplicativo", "vc", "pra", "to", "os", "vcs", "nao", "pq", "mim", "ai", "ta", "ja", "ter", "fazer", "lá", "deu", "dado", "então", "vou", "vai", "veze", "ficar", "tá", "apena"])


In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Criando o objeto wordcloud com as configs necessárias. Cor escolhida = preta, origem dos dados = all_content
wordcloud = WordCloud(stopwords=stopwords,
                      background_color='black', width=1600,                            
                      
                      height=800).generate(all_content)

#configurando forma de apresentação do gráfico e apresentando no notebook.
fig, ax = plt.subplots(figsize=(16,8))            
ax.imshow(wordcloud, interpolation='bilinear')       
ax.set_axis_off()
plt.imshow(wordcloud)